In [1]:
import tensorflow as tf
import numpy as np
import os
import re

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


In [2]:
# PARAMETERS
img_width, img_height = 224, 224
image_size = img_width* img_height
LEARNING_RATE = 0.0005
EPSILON = 1e-4
LABEL_NUM = 2
TRAIN_DATASET = 1000
TEST_DATASET = 2500
BATCH_SIZE = 32
epoch = 50

In [3]:
# Get data directory
def read_data(data_dir):
    data_directory = data_dir
    
    filename ={'train':[],'validation':[]}
    dataset = ['train','validation']

    for dataset_name in dataset:
        for subFile in os.listdir(data_directory):
            if dataset_name in subFile and re.match(r'^.*\.tfrecords$', subFile):
                filename[dataset_name].append(data_directory+"/"+subFile)
#     print(filename)        
#     training_filenames = filename['train']
#     validation_filenames = filename['validation']
    
    return filename['train'] , filename['validation'] 

In [4]:
def parse(serialized):
    features = { 'label': tf.FixedLenFeature([], tf.int64),
               'text_label': tf.FixedLenFeature([], tf.string),
               'image': tf.FixedLenFeature([], tf.string)
               }

    parsed_example = tf.parse_single_example(serialized=serialized,
                                             features=features)

    image_raw = parsed_example['image']

    image = tf.decode_raw(image_raw, tf.uint8)
    image = tf.reshape(image, [img_width, img_height, 3])
#     image = tf.image.resize_images(
#                 image,
#                 [64, 64]
#             )
#     image = tf.image.rgb_to_grayscale(image, name=None)
    image = tf.cast(image, tf.float32)    
    image = image / 255.
    
    label = parsed_example['label']
    label = tf.one_hot(label,LABEL_NUM)
    label = tf.cast(label, tf.float32)
#     label = tf.reshape(label, [1])
    
    return image, label

In [5]:
def input_pipeline(filename , BUFFER_SIZE = 25000):
    dataset = tf.data.TFRecordDataset(filename)
    dataset = dataset.map(parse)
    dataset = dataset.shuffle(buffer_size=BUFFER_SIZE)
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    iterator = dataset.make_one_shot_iterator()
    return iterator


In [6]:
class CNN():
    def __init__(self):
        self.cost = None
        self.accuracy = None
        self.prediction = None
        self.logits = None
        self.global_step = None 
        
        
#         print("CNN")

    def predict(self, x):
        
        #model VGG16
        x = tf.layers.conv2d(inputs = x,filters = 32,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu) 
#         x = tf.layers.conv2d(inputs = x,filters = 64,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.max_pooling2d(x, pool_size = [2,2], strides =[2,2])

#         x = tf.layers.conv2d(inputs = x,filters = 128,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.conv2d(inputs = x,filters = 64,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
        x = tf.layers.max_pooling2d(x, pool_size = [2,2], strides =[2,2])
        
#         x = tf.layers.conv2d(inputs = x,filters = 256,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
#         x = tf.layers.conv2d(inputs = x,filters = 256,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
#         x = tf.layers.conv2d(inputs = x,filters = 128,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
#         x = tf.layers.max_pooling2d(x, pool_size = [2,2], strides =[2,2])


        x = tf.layers.flatten(x)
        x = tf.layers.dense(x, 128, activation=tf.nn.relu)       
        x = tf.layers.dropout(x,rate=0.5)
        x = tf.layers.dense(x, 2, activation=tf.nn.sigmoid, use_bias = True)
        
        return x
    
    def train(self, x, y, learning_rate = 0.01, epsilon = 1e-10):
        self.logits = self.predict(x)
        
        with tf.name_scope("cost"):
            self.cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = y, logits = self.logits)  , name='cost')
            tf.summary.scalar("cost", self.cost)
        
        self.global_step = tf.train.get_or_create_global_step()   
        optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate, epsilon=epsilon)
        optimize = optimizer.minimize(self.cost , global_step = self.global_step)
        accuracy = self.validate(self.logits, y)
        return self.cost, accuracy, optimize, self.global_step
        
    def validate(self, logits, y):         
        with tf.name_scope("validate"):
            correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            tf.summary.scalar("accuracy", self.accuracy )
            tf.summary.histogram("accuracy", self.accuracy )
            
        return self.accuracy
    
    def test(self, x, y):
        self.logits = self.predict(x)
        accuracy = validate(self.logits, y)
        return accuracy
        

 

In [7]:
# x, y = train_dataset.get_next()
# x = tf.layers.conv2d(inputs = x,filters = 32,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu) 
# x = tf.layers.max_pooling2d(x, pool_size = 2, strides = 2)
# x = tf.layers.conv2d(inputs = x,filters = 32,kernel_size = [3,3], strides=1, padding='SAME', activation=tf.nn.relu)
# x = tf.layers.max_pooling2d(x, pool_size = [2,2], strides =[2,2])

# x = tf.layers.conv2d(inputs = x,filters = 64,kernel_size = [5,5], strides=1, padding='SAME', activation=tf.nn.relu)
# x = tf.layers.max_pooling2d(x, pool_size = [2,2], strides =[2,2])

# x = tf.layers.conv2d(inputs = x,filters = 128,kernel_size = [5,5], strides=1, padding='SAME', activation=tf.nn.relu)
# x = tf.layers.max_pooling2d(x, pool_size = [2,2], strides =[2,2])

# x = tf.layers.flatten(x)
# x = tf.layers.dense(x, 128, activation=tf.nn.relu)
# x = tf.layers.dropout(x,rate=0.1)
# x = tf.layers.dense(x, 2, activation=tf.nn.sigmoid, use_bias = True)

# logits = x
# tf.summary.histogram("logits", logits)
# with tf.name_scope("cost"):
#     cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = y, logits = logits)  , name='cost')
#     tf.summary.scalar("cost", cost)
    
# global_step = tf.train.get_or_create_global_step()   
# with tf.name_scope("train"):
#     optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE, epsilon = EPSILON)
#     optimize = optimizer.minimize(cost , global_step = global_step)
    
# with tf.name_scope("validate"):
#     correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     tf.summary.scalar("accuracy", accuracy)
#     tf.summary.histogram("accuracy", accuracy)

# merge = tf.summary.merge_all()

In [8]:
tf.reset_default_graph()

In [9]:
data_dir = "../data"    
training_filenames, validation_filenames= read_data(data_dir)
train_dataset = input_pipeline(training_filenames)
test_dataset = input_pipeline(validation_filenames)

print(training_filenames)

['../data/train-3.tfrecords', '../data/train-2.tfrecords', '../data/train-1.tfrecords', '../data/train-0.tfrecords']


In [10]:
clf = CNN()
x_train, y_train = train_dataset.get_next()
x_test, y_test = train_dataset.get_next()

cost , accuracy, optimize, global_step = clf.train(x = x_train, y = y_train, learning_rate = LEARNING_RATE, epsilon = EPSILON)
test_accuracy = clf.validate(clf.predict(x_test), y_test)
merge = tf.summary.merge_all()

In [11]:
ACC_TEST , ACC_TRAIN = [],[]
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter('./logs/train', sess.graph)
    test_writer =  tf.summary.FileWriter('./logs/test', sess.graph)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    for i in range(epoch):        
        total_loss, total_acc = 0 ,0
        for x in range(0, (TRAIN_DATASET//BATCH_SIZE)* BATCH_SIZE, BATCH_SIZE):            
            if x == 0:
                loss, acc, _, summary, global_step_val= sess.run([cost, accuracy, optimize, merge, global_step])
                train_writer.add_summary(summary, global_step_val)    
            else:
                loss, acc, _= sess.run([cost, accuracy, optimize])
                
            total_loss += loss
            total_acc += acc    
        

        total_loss /= (TRAIN_DATASET // BATCH_SIZE) 
        total_acc /= (TRAIN_DATASET // BATCH_SIZE)
        ACC_TRAIN.append(total_acc)
        
        total_acc = 0 
        for x in range(0, (TEST_DATASET//BATCH_SIZE)* BATCH_SIZE, BATCH_SIZE):
            acc, summary, global_step_val= sess.run([test_accuracy, merge, global_step])
            test_writer.add_summary(summary, global_step_val)   
                
            total_acc += acc    
        
        total_acc /= (TEST_DATASET // BATCH_SIZE)
        ACC_TEST.append(total_acc)
        print("Epoch : %d | TRAIN Loss %f , Accuracy %f | TEST Accuracy %f"% ((i+1),total_loss, ACC_TRAIN[-1], ACC_TEST[-1]))  
        

Epoch : 1 | TRAIN Loss 0.695614 , Accuracy 0.518145 | TEST Accuracy 0.500801
Epoch : 2 | TRAIN Loss 0.693119 , Accuracy 0.516129 | TEST Accuracy 0.506010
Epoch : 3 | TRAIN Loss 0.693078 , Accuracy 0.521169 | TEST Accuracy 0.506410
Epoch : 4 | TRAIN Loss 0.693035 , Accuracy 0.498992 | TEST Accuracy 0.496795
Epoch : 5 | TRAIN Loss 0.692930 , Accuracy 0.486895 | TEST Accuracy 0.520433
Epoch : 6 | TRAIN Loss 0.692391 , Accuracy 0.502016 | TEST Accuracy 0.503205
Epoch : 7 | TRAIN Loss 0.693122 , Accuracy 0.489919 | TEST Accuracy 0.502003
Epoch : 8 | TRAIN Loss 0.693132 , Accuracy 0.503024 | TEST Accuracy 0.520032
Epoch : 9 | TRAIN Loss 0.693098 , Accuracy 0.518145 | TEST Accuracy 0.511218
Epoch : 10 | TRAIN Loss 0.693002 , Accuracy 0.500000 | TEST Accuracy 0.517628
Epoch : 11 | TRAIN Loss 0.692878 , Accuracy 0.480847 | TEST Accuracy 0.512821
Epoch : 12 | TRAIN Loss 0.688677 , Accuracy 0.534274 | TEST Accuracy 0.526843
Epoch : 13 | TRAIN Loss 0.691075 , Accuracy 0.473790 | TEST Accuracy 0.51